In [35]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ai-201-b-mse-1-ai-d/sample_submission.csv
/kaggle/input/ai-201-b-mse-1-ai-d/train.csv
/kaggle/input/ai-201-b-mse-1-ai-d/test.csv


In [36]:
#Import Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor


In [37]:
#Load the Dataset
train=pd.read_csv('/kaggle/input/ai-201-b-mse-1-ai-d/train.csv')
test=pd.read_csv('/kaggle/input/ai-201-b-mse-1-ai-d/test.csv')
train.head()

,Input_Voltage_Deviation,Current_Flow_Variance,Thermal_Load_Index,Power_Factor_Offset,Energy_Output_Difference
0,0.735,0.342,0.567,2.837,848.85
1,-2.167,0.762,0.043,-2.262,-43.25
2,-0.277,0.923,-0.831,0.847,93.85
3,0.842,-0.941,0.589,0.198,98.95
4,-1.503,0.872,0.166,-0.541,-67.55


In [38]:
#Filling Missing Values
for x in train.columns:
    train[x]=train[x].fillna(train[x].mode()[0])
for x in test.columns:
    test[x]=test[x].fillna(test[x].mode()[0])    

In [39]:
#Drop ID Column(if present)
id_column=None
if 'id' in train.columns:
    train=train.drop('id',axis=1)
if 'id' in test.columns:
    id_column=test['id']
    test=test.drop('id',axis=1)

In [40]:
#Features and Target Column
x=train.drop('Energy_Output_Difference',axis=1)
y=train['Energy_Output_Difference']

In [41]:
#Spliting into Training and Validiation
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.2,random_state=0)

In [42]:
#Models
models={
    'Random Forest': RandomForestRegressor(n_estimators=200,random_state=0),
    'Decision Tree': DecisionTreeRegressor(random_state=0),
    'Linear Regression': LinearRegression(),
    'XGBoost': XGBRegressor(
        n_estimators=500,
        learning_rate=0.05,
        max_depth=5,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=0,
        verbosity=0
    )
}

In [43]:
#Fit , Predict and Evaluate the best model
r2_scores={}
for name,model in models.items():
    model.fit(x_train,y_train)
    y_pred=model.predict(x_val)
    r2=r2_score(y_val,y_pred)
    r2_scores[name]=r2
    print(f'{name} R2 Score: {r2:.4f}')

best_model_name=max(r2_scores,key=r2_scores.get)
best_model=models[best_model_name]
print("Best Model Name: ",best_model_name)

Random Forest R2 Score: 0.7705
Decision Tree R2 Score: 0.6486
Linear Regression R2 Score: 0.5621
XGBoost R2 Score: 0.7966
Best Model Name:  XGBoost


In [44]:
#On Testing Data
x_test=test.copy()
y_pred_test=best_model.predict(x_test)

In [47]:
#Generating submission file
submission=pd.DataFrame({
    'id':id_column,
    'Energy_Output_Difference':y_pred_test
})
submission.to_csv("submission1.csv",index=False)
print(f"The best model is {best_model_name} with R2 Score = {r2_scores[best_model_name]:.4f}")
print("submission1.csv generated using best model!")
submission.head()

The best model is XGBoost with R2 Score = 0.7966
submission1.csv generated using best model!


,id,Energy_Output_Difference
0,1,-6.468377
1,2,83.861382
2,3,54.188915
3,4,3.415136
4,5,-83.812889
